# Notes
/!\ This notebook is fully functional on Google Colab./!\


# Reinforcement Learning in Finite MDPs

In [0]:
!git clone https://github.com/rlgammazero/mvarl_hands_on.git > /dev/null 2>&1

In [0]:
!pip install gym

## MDPs

In [0]:
import sys
sys.path.insert(0, './mvarl_hands_on/utils')
import numpy as np
from scipy.special import softmax # for SARSA
import matplotlib.pyplot as plt
import json
import math
from cliffwalk import CliffWalk
from test_env import ToyEnv1
import gridworld

Setting up the environment

In [0]:
env = CliffWalk(proba_succ=0.98)

####################################################################################
# You probably want to test smaller enviroments before
#env = ToyEnv1(gamma=0.99)
####################################################################################

# Useful attributes
print("Set of states:", env.states)
print("Set of actions:", env.actions)
print("Number of states: ", env.Ns)
print("Number of actions: ", env.Na)
print("P has shape: ", env.P.shape)  # P[s, a, s'] = env.P[s, a, s']
print("discount factor: ", env.gamma)
print("")

# Usefult methods
state = env.reset() # get initial state
print("initial state: ", state)
print("reward at (s=1, a=3,s'=2): ", env.reward_func(1,3,2))
print("")

# A random policy
policy = np.random.randint(env.Na, size = (env.Ns,))
print("random policy = ", policy)

# Interacting with the environment
print("(s, a, s', r):")
for time in range(4):
    action = policy[state]
    next_state, reward, done, info = env.step(action)
    print(state, action, next_state, reward)
    if done:
        break
    state = next_state
print("")
print(env.R.shape)

## Question 1: Value iteration
1. Write a function applying the optimal Bellman operator on a provided Q function: $Q_1 = LQ_0, \; Q_0\in \mathbb{R}^{S\times A}$
2. Write a function implementing Value Iteration (VI) with $\infty$-norm stopping condition (reuse function implemented in 1)
3. Evaluate the convergence of your estimate, i.e., plot the value $\|Q_n - Q^\star\|_{\infty} = \max_{s,a} |Q_n(s,a) - Q^\star(s,a)|$

In [0]:
# --------------
# Point 1
# --------------
def bellman_operator(Q0, Ns, Na, R, P, gamma):
    
    # Computing Q1 = LQ0 #
    Q1 = np.zeros((Ns,Na))
    greedy_policy = np.zeros(Ns)
    for s in range(Ns):
        for a in range(Na):
            Q1[s,a] = np.dot(P[s,a,:], R[s,a,:]) + gamma * np.dot(P[s,a,:], np.max(Q0, axis = 1))
            
    # Computing the greedy policy #
    for s in range(Ns):
        greedy_policy[s] = np.argmax(Q1[s,:])
        
    return Q1, greedy_policy

In [0]:
# --------------
# Point 2
# --------------
def value_iteration(Q0, env, epsilon=1e-6):
    gamma = env.gamma
    Q_history = [Q0]
    err = epsilon + 1
    while err >= epsilon:
        Q0, greedy_policy = bellman_operator(Q0, env.Ns, env.Na, env.R, env.P, gamma)
        err = np.max(np.abs(Q0 - Q_history[-1]))
        Q_history.append(Q0)

    return Q0, greedy_policy, Q_history

In [0]:
# --------------
# Point 3
# --------------


with open("./mvarl_hands_on/data/Q_opts.json", "r") as fp:
    Qopts = json.load(fp)
Qstar = Qopts["{}_{}".format(type(env).__name__,env.gamma)]

Q0 = np.zeros((env.Ns,env.Na))
Q, greedy_policy, Q_history = value_iteration(Q0, env, epsilon = 1e-8)

norm_values = [np.max(np.abs(Q_history[i] - Qstar)) for i in range(len(Q_history))]

plt.plot(norm_values)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Q-learning: Convergence of Q")

## Question 2: Q learning
Q learning is a model-free algorithm for estimating the optimal Q-function online.
It is an off-policy algorithm since the samples are collected with a policy that is (potentially) not the one associated to the estimated Q-function.

1. Implement Q learning with $\epsilon$-greedy exploration.
  - Plot the error in Q-functions over iterations
  - Plot the sum of rewards as a function of iteration


$\epsilon$-greedy policy:
$$
\pi(s) = \begin{cases}
\max_a Q(s,a) & \text{w.p.} \epsilon\\
\text{random action} & \text{w.p.} 1- \epsilon
\end{cases}
$$

#Notes

The learning rate $\alpha_{s,a}(t)$ , to ensure convergence, should satistify the Robbins-Monro conditions, *i.e.*
$$
\sum_{t} \alpha_{s,a}(t) = \infty, \quad \sum_{t} \alpha_{s,a}(t)^2 < \infty.
$$
In our algorithm, we coud have set, for instance, $\alpha_{s,a}(t) = 1/(N_t(s,a))^{1/2 + \varepsilon}$ with $\varepsilon > 0$ and $N_t(s,a)$ the number of times the state-action $(s,a)$ has been visited at time $t$. Nonetheless, we have decided to put 
$$
\alpha_{s,a}(t) := \frac{1}{\log( e + \log(1 + N_t(s,a))} \quad (1).
$$
This learning rate **does not** satisfy Robbins-Monro conditions, so technically, nothing ensure convergence ! That being said, when we tested our code with a learning rate that did verify those conditions, we saw that the convergence was very slow ! This might be explained by the fact that we started quite far away from the optimal solution. The learning rate (1) has the advantage of decreasing very slowly, so that we take enough big steps at the beginning of the learning process, and therefore we experimentally observe a better approximation of the true $Q^\star$. The drawback is that, near the optimum, we do observe fluctuations (*i.e.* the gap, after decreasing, rebounces)  for the learning rate does not decay quickly enough. 

In [0]:
# ---------------------------
# Q-Learning
# ---------------------------
# suggested interface
# you can change it!

class QLearning:
    """
    Q learning with epsilon-greedy exploration
    """
    def __init__(self, env):
        self.env = env 
        self.Q = np.zeros((env.Ns, env.Na))
        self.count = np.zeros((env.Ns, env.Na))
        
    def learning_rate(self, state, action):
        return 1/np.log(np.e + np.log(1 + self.count[state, action]))
        
    def sample_action(self, state, greedy):
        h = np.random.rand()
        if h < greedy:
            return np.argmax(self.Q[state, :])
        else:
            return np.random.randint(env.Na)
    
    def update(self, state, action, next_state, reward):
        self.count[state, action] += 1
        lr = self.learning_rate(state, action)
        self.Q[state,action] = (1 - lr) * self.Q[state, action] \
                            + lr * (reward + self.env.gamma * np.max(self.Q[next_state,:]))
        
    

In [0]:
# --------------
# Point 1
# --------------
# Number of Q learning steps
max_steps = int(1e5)  
# max_steps = 10

Q0 = np.zeros((env.Ns, env.Na))
# Use the previous code to verify the correctness of q learning
Q_opt, pi_opt, Q_hist = value_iteration(Q0, env, epsilon=1e-8)


# main algorithmic loop
env.reset()
state = env.state
ql = QLearning(env)
greedy = 0.2
norm_values = []
cum_rewards = [0.]
t = 0

while t < max_steps:
    action = ql.sample_action(state, greedy)
    observation, reward, done, info = env.step(action)
    ql.update(state, action, observation, reward)
    state = observation
    norm_values.append(np.abs(ql.Q - Q_opt).mean())
    cum_rewards.append(env.gamma * cum_rewards[-1] + reward)
    t = t + 1
    
print(env.render())
print("optimal policy: ", pi_opt)
greedy_policy = np.argmax(ql.Q, axis=1)
print("est policy:", greedy_policy)


fig = plt.figure(figsize=(10, 4))
plt.subplot(121)

plt.semilogy(norm_values)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Q-learning: Convergence of Q")

plt.subplot(122)
plt.plot(cum_rewards, c='r')
plt.xlabel('Iteration')
plt.title("Cumulative reward")
